In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import plotly.graph_objects as go
from plotly import offline as pyoff
RANDOM_STATE = 42
from datetime import date
import calendar
import datetime as dt
from datetime import datetime

In [2]:
df = pd.read_csv('df_banc_rev.csv')

In [3]:
###Para obtener el listado de los 10000 mejores clientes, a quienes enviar el email, vamos a seguir el criterio de
###los que han hecho mas operaciones de compra con nosotros; preferimos este criterio, en lugar del revenue, porque
##teniendo en cuenta el alto nivel de abandono (archivo Churn2) que tenemos, es mas facil fidelizar y a su vez, con un
##porcentaje de interes frente a nuestra mailingm aquel cliente,
## que nos compra mas veces, y ha experimentado  nuestro servicio de atencio al cliente,
###que el que solo lo hace una vez, y nos pueda dar más revenue.

In [4]:
df_top = df.groupby(by=['pk_cid','entry_date'],as_index=False)["unidades_vendidas_total"].count()

In [5]:
df_top

,pk_cid,entry_date,unidades_vendidas_total
0,15891,2018-07-28,2
1,16063,2018-11-19,7
2,16203,2018-12-23,6
3,16502,2018-09-30,9
4,17457,2017-09-16,17
...,...,...,...
454901,1553685,2019-05-31,1
454902,1553686,2019-05-31,1
454903,1553687,2019-05-31,1
454904,1553688,2019-05-31,1


In [6]:
df_top = df_top[df_top['unidades_vendidas_total']==17]

In [7]:
df_top.describe()

,pk_cid,unidades_vendidas_total
count,2.328910e+05,232891.0
mean,1.178941e+06,17.0
std,1.261654e+05,0.0
min,1.745700e+04,17.0
25%,1.086818e+06,17.0
50%,1.180961e+06,17.0
75%,1.280713e+06,17.0
max,1.379128e+06,17.0


In [8]:
NOW = dt.datetime(2019,5,28)

In [9]:
df['entry_date'] = pd.to_datetime(df['entry_date'])

In [10]:
df['pk_partition'] = pd.to_datetime(df['pk_partition'])

In [11]:
df_17m = df[(df.entry_date.dt.date < date(2019,5,28)) & (df.entry_date.dt.date >= date(2018,1,1))].reset_index(drop=True)

In [12]:
df_frequency = df_17m.groupby('pk_cid').entry_date.count().reset_index()
df_frequency.columns = ['pk_cid','Frequency']

In [13]:
df = pd.merge(df,df_frequency,on='pk_cid',how='right')

In [14]:
df.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Unnamed: 0,2168122,1479563,5134317,2962973,4028169,5523397,4876040,3628236,4480637,5134321,5523396,3628238,4028170,4876039,4480636,4876038,3628237,2171490,5523395,5134320
pk_cid,15891,15891,16063,16063,16063,16063,16063,16063,16063,16203,16203,16203,16203,16203,16203,16502,16502,16502,16502,16502
pk_partition,2018-08-28 00:00:00,2018-07-28 00:00:00,2019-04-28 00:00:00,2018-11-28 00:00:00,2019-01-28 00:00:00,2019-05-28 00:00:00,2019-03-28 00:00:00,2018-12-28 00:00:00,2019-02-28 00:00:00,2019-04-28 00:00:00,2019-05-28 00:00:00,2018-12-28 00:00:00,2019-01-28 00:00:00,2019-03-28 00:00:00,2019-02-28 00:00:00,2019-03-28 00:00:00,2018-12-28 00:00:00,2018-09-28 00:00:00,2019-05-28 00:00:00,2019-04-28 00:00:00
entry_date,2018-07-28 00:00:00,2018-07-28 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-11-19 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-12-23 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00
active_customer,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
revenue_payroll,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
revenue_payroll_account,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
revenue_total,0,10,0,0,0,0,0,0,0,10,10,0,10,10,10,20,10,0,20,20
unidades_vendidas_total,0,1,0,0,0,0,0,0,0,1,1,0,1,1,1,2,1,0,2,2


In [15]:
rfmTable = df.groupby('pk_cid').agg({'entry_date': lambda x: (NOW - x.max()).days,'Frequency': lambda x: len(x), 'revenue_total': lambda x: x.sum()})
rfmTable['entry_date'] = rfmTable['entry_date'].astype(int)
rfmTable.rename(columns={'entry_date': 'recency', 
                         'Frequency': 'frequency',
                         'revenue_total': 'monetary_value'}, inplace=True)

In [16]:
rfmTable

,recency,frequency,monetary_value
pk_cid,,,
15891,304,2,10.0
16063,190,7,0.0
16203,156,6,50.0
16502,240,9,130.0
17799,509,17,170.0
...,...,...,...
1553111,1,1,0.0
1553112,1,1,0.0
1553113,1,1,0.0


In [17]:
first_customer=df[df['pk_cid']==16502]

In [18]:
first_customer.T

,15,16,17,18,19,20,21,22,23
Unnamed: 0,4876038,3628237,2171490,5523395,5134320,4480638,4028330,2962972,2943633
pk_cid,16502,16502,16502,16502,16502,16502,16502,16502,16502
pk_partition,2019-03-28 00:00:00,2018-12-28 00:00:00,2018-09-28 00:00:00,2019-05-28 00:00:00,2019-04-28 00:00:00,2019-02-28 00:00:00,2019-01-28 00:00:00,2018-11-28 00:00:00,2018-10-28 00:00:00
entry_date,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00,2018-09-30 00:00:00
active_customer,1,1,0,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...
revenue_payroll,0,0,0,0,0,0,0,0,0
revenue_payroll_account,0,0,0,0,0,0,0,0,0
revenue_total,20,10,0,20,20,20,20,10,10
unidades_vendidas_total,2,1,0,2,2,2,2,1,1


In [19]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [20]:
segmented_rfm = rfmTable

In [21]:
###The lowest recency, highest frequency and monetary amounts are our best customers.

In [22]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [23]:
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [24]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
pk_cid,,,,,,
15891,304,2,10.0,4,4,3
16063,190,7,0.0,1,3,4
16203,156,6,50.0,1,4,3
16502,240,9,130.0,2,2,1
17799,509,17,170.0,4,1,1


In [25]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)


In [26]:
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,
15891,304,2,10.0,4,4,3,443
16063,190,7,0.0,1,3,4,134
16203,156,6,50.0,1,4,3,143
16502,240,9,130.0,2,2,1,221
17799,509,17,170.0,4,1,1,411


In [27]:
##It is obvious that the first customer is not our best customer at all.

In [28]:
segmented_rfm.count()

recency           153232
frequency         153232
monetary_value    153232
r_quartile        153232
f_quartile        153232
m_quartile        153232
RFMScore          153232
dtype: int64

In [29]:
segmented_rfm['RFMScore'].value_counts()

322    21880
233    20623
411    17229
143    14504
144    13551
234    11587
412    11310
414     5902
324     5263
321     4702
141     4472
231     3521
133     2868
323     2736
134     1764
244     1554
344     1421
413     1387
444     1201
223      924
142      810
312      707
131      631
343      406
243      358
443      334
232      231
441      149
224      129
431      126
421      123
433      115
221       99
132       83
333       82
311       80
423       66
314       61
341       48
331       36
434       30
313       30
241       25
424       18
422       17
334       15
442       10
432        6
222        3
342        3
242        1
332        1
Name: RFMScore, dtype: int64

In [30]:
###Como tenemos la urgente necesidad de incrementar los ingresos, nos enfocaremos, a aquellos clientes, con mayor facturacion,
###y en un periodo no muy alto ,entre compra y compra, ya que pueden ser más fieles y ser mas receptivos a nuestra propuesta 
###por email.Como por desgracia, solo son 4500 clientes, los restantes 5500, los basaremos en los que nos han dado 
##jla mayor facturacion, pero hace tiempo que no compran,Cogeremos los clientes con un RFM  141 y 411, llegando a unos 10000 clientes ,nuestro mailing

In [31]:
df_best = segmented_rfm[segmented_rfm['RFMScore']=='141'].sort_values('monetary_value', ascending=False).head(4472)

In [32]:
df_best

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,
1517456,161,6,910.0,1,4,1,141
1512063,178,6,900.0,1,4,1,141
1515010,169,6,900.0,1,4,1,141
1520203,148,6,840.0,1,4,1,141
1518210,157,6,810.0,1,4,1,141
...,...,...,...,...,...,...,...
1526351,127,5,120.0,1,4,1,141
1536961,87,3,120.0,1,4,1,141
1517519,160,6,120.0,1,4,1,141


In [33]:
df_best1= segmented_rfm[segmented_rfm['RFMScore']=='411'].sort_values('monetary_value', ascending=False).head(5500)

In [34]:
df_best1

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,
1374819,505,17,4290.0,4,1,1,411
1392320,408,14,3190.0,4,1,1,411
1377496,492,17,2880.0,4,1,1,411
1374123,508,17,2850.0,4,1,1,411
1374403,506,17,2840.0,4,1,1,411
...,...,...,...,...,...,...,...
1401318,351,12,640.0,4,1,1,411
1397743,374,13,640.0,4,1,1,411
1397057,379,12,640.0,4,1,1,411


In [35]:
df_best_total = pd.concat([df_best,df_best1],axis=0)

In [36]:
df_best_total

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,
1517456,161,6,910.0,1,4,1,141
1512063,178,6,900.0,1,4,1,141
1515010,169,6,900.0,1,4,1,141
1520203,148,6,840.0,1,4,1,141
1518210,157,6,810.0,1,4,1,141
...,...,...,...,...,...,...,...
1401318,351,12,640.0,4,1,1,411
1397743,374,13,640.0,4,1,1,411
1397057,379,12,640.0,4,1,1,411


In [37]:
df_best_total['RFMScore'].value_counts()

411    5500
141    4472
Name: RFMScore, dtype: int64

In [38]:
df['region_code']

0          28.0
1          28.0
2          28.0
3          28.0
4          28.0
           ... 
1250311     8.0
1250312    28.0
1250313    21.0
1250314    28.0
1250315    28.0
Name: region_code, Length: 1250316, dtype: float64

In [39]:
df['age']

0          59
1          59
2          62
3          62
4          62
           ..
1250311    28
1250312    43
1250313    46
1250314    76
1250315    70
Name: age, Length: 1250316, dtype: int64

In [40]:
df['salary']

0          113652.32033
1          113652.32033
2          113652.32033
3          113652.32033
4          113652.32033
               ...     
1250311    113652.32033
1250312    113652.32033
1250313    113652.32033
1250314    113652.32033
1250315    113652.32033
Name: salary, Length: 1250316, dtype: float64

In [53]:
df_best_total

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
pk_cid,,,,,,,
1517456,161,6,910.0,1,4,1,141
1512063,178,6,900.0,1,4,1,141
1515010,169,6,900.0,1,4,1,141
1520203,148,6,840.0,1,4,1,141
1518210,157,6,810.0,1,4,1,141
...,...,...,...,...,...,...,...
1401318,351,12,640.0,4,1,1,411
1397743,374,13,640.0,4,1,1,411
1397057,379,12,640.0,4,1,1,411


In [54]:
df_best_features = df[['pk_cid','unidades_vendidas_total','region_code','salary','age','gender','em_acount','debit_card']]

In [43]:
df_best_features

,pk_cid,unidades_vendidas_total,region_code,salary,age,gender,em_acount,debit_card
0,15891,0.0,28.0,113652.32033,59,1,0,0
1,15891,1.0,28.0,113652.32033,59,1,1,0
2,16063,0.0,28.0,113652.32033,62,1,0,0
3,16063,0.0,28.0,113652.32033,62,1,0,0
4,16063,0.0,28.0,113652.32033,62,1,0,0
...,...,...,...,...,...,...,...,...
1250311,1553111,0.0,8.0,113652.32033,28,1,0,0
1250312,1553112,0.0,28.0,113652.32033,43,1,0,0
1250313,1553113,0.0,21.0,113652.32033,46,2,0,0
1250314,1553114,0.0,28.0,113652.32033,76,2,0,0


In [55]:
df_best_definitivo = pd.merge(df_best_total, df_best_features ,on='pk_cid',how='inner')

In [56]:
df_best_definitivo

,pk_cid,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore,unidades_vendidas_total,region_code,salary,age,gender,em_acount,debit_card
0,1517456,161,6,910.0,1,4,1,141,5.0,28.0,113652.32033,37,2,0,1
1,1517456,161,6,910.0,1,4,1,141,5.0,28.0,113652.32033,37,2,0,1
2,1517456,161,6,910.0,1,4,1,141,7.0,28.0,113652.32033,37,2,0,1
3,1517456,161,6,910.0,1,4,1,141,6.0,28.0,113652.32033,37,2,0,1
4,1517456,161,6,910.0,1,4,1,141,1.0,28.0,113652.32033,37,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100026,1382256,470,16,640.0,4,1,1,411,1.0,28.0,104109.42000,33,1,0,0
100027,1382256,470,16,640.0,4,1,1,411,1.0,28.0,104109.42000,33,1,0,0
100028,1382256,470,16,640.0,4,1,1,411,1.0,28.0,104109.42000,33,1,0,0
100029,1382256,470,16,640.0,4,1,1,411,1.0,28.0,104109.42000,34,1,0,0


In [57]:
df_best_total['monetary_value']

pk_cid
1517456    910.0
1512063    900.0
1515010    900.0
1520203    840.0
1518210    810.0
           ...  
1401318    640.0
1397743    640.0
1397057    640.0
1399722    640.0
1382256    640.0
Name: monetary_value, Length: 9972, dtype: float64

In [47]:
##estudio de la region

In [58]:
df_best_unidades_vendidas = df_best_definitivo.groupby('region_code')['unidades_vendidas_total'].sum().reset_index()

In [59]:
df_best_unidades_vendidas.sort_values(by = ["unidades_vendidas_total"], ascending = False, inplace = True)

In [60]:
df_best_unidades_vendidas

,region_code,unidades_vendidas_total
27,28.0,73333.0
7,8.0,17404.0
45,46.0,11823.0
40,41.0,9657.0
2,3.0,8152.0
28,29.0,7183.0
10,11.0,5859.0
29,30.0,5773.0
34,35.0,5100.0
14,15.0,5059.0


In [61]:
plot_data = [
    go.Bar(
        x=df_best_unidades_vendidas['region_code'],
        y=df_best_unidades_vendidas['unidades_vendidas_total']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Unidades vendidas Best por Region'
    )
plt.figure(figsize=(16,16))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig500.html')
pyoff.iplot(fig)

<Figure size 1152x1152 with 0 Axes>

In [62]:
df_best_monetary_value = df_best_definitivo.groupby('region_code')['monetary_value'].sum().reset_index()

In [63]:
df_best_monetary_value.sort_values(by = ["monetary_value"], ascending = False, inplace = True)

In [64]:
plot_data = [
    go.Bar(
        x=df_best_monetary_value['region_code'],
        y=df_best_monetary_value['monetary_value']
    )
]

plot_layout = go.Layout(
        xaxis={"type": "category"},
        title='Ventas Best por Region'
    )
plt.figure(figsize=(16,16))
fig = go.Figure(data=plot_data, layout=plot_layout)
fig.write_html('fig501.html')
pyoff.iplot(fig)

<Figure size 1152x1152 with 0 Axes>